# Морозова Дарья, ИАД-2, дз 3

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

In [47]:
data = pd.read_csv("train_subset.csv", index_col='id')
data.head()

,title,description,Category_name,Category
id,,,,
382220,Прихожая,В хорошем состоянии. Торг,Мебель и интерьер,20
397529,Кордиант 215/55/16 Летние,Кордиант 215/55/16 Летние/\n /\nАртикул: 1737l...,Запчасти и аксессуары,10
584569,Стол,"Стол, 2 рабочих места . Стол серого цвета, в д...",Мебель и интерьер,20
2513100,Комбинезон,Размер-42/44,"Одежда, обувь, аксессуары",27
1091886,Ветровка,На 2 года,Детская одежда и обувь,29


In [3]:
data.shape

(30000, 4)

In [48]:
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()
del data

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train[:5]

array([['Сапоги 46 размер новые', 'Сапоги 46 размер новые'],
       ['Светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку. В эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iPhone 7 plus 128GB Red красный в наличии',
        '\xa0/\n/\n Данная цена только для подписчиков Instagram: iQmac/\n/\n Новый красный айфон 7 Plus в наличии это элегантный и мощный смартфон, который готов в полной мере раскрыть новые возможности iOS 10. Аппарат с 4-ядерным процессором А10 и 3 ГБ ОЗУ с легкостью решает самые ресурсоемкие задачи, позволяя наслаждаться быстродействием «тяжелых» приложений и игр на 5,5-дюймовом дисплее. Аппарат получил экран, как у iPad Pro, так что картинка теперь соответствует кинематографическому стандарту.'],
       ['Пион Ирис Ромашка рассада',
        'Пион куст 500 р ( более 10 шт)/\nСаженец/ корень 100р/\nРастут у нас более 70 лет/\nРозовые, бордовые и белые/\nНа фото цветы 2018г/\nП. Зубчанинов

In [6]:
X_train.shape

(21000, 2)

### Токенизация

Токенизировать title и description в train и test

In [50]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

def preprocess(text: str) -> str:
    return ' '.join(tokenizer.tokenize(text.lower()))

#Токенизация для X_train и X_test циклами по всем элементам
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        X_train[i][j] = preprocess(X_train[i][j])
for i in range(X_test.shape[0]):
    for j in range(X_test.shape[1]):
        X_test[i][j] = preprocess(X_test[i][j])

In [51]:
X_train[:10]

array([['сапоги 46 размер новые', 'сапоги 46 размер новые'],
       ['светильники потолочный swarovski',
        'светильники потолочные swarovski 6 штук , цена за штуку . в эксплуатации 2 года , продаются в связи со сменой интерьера в квартире'],
       ['iphone 7 plus 128gb red красный в наличии',
        '/ / данная цена только для подписчиков instagram : iqmac / / новый красный айфон 7 plus в наличии это элегантный и мощный смартфон , который готов в полной мере раскрыть новые возможности ios 10 . аппарат с 4 - ядерным процессором а10 и 3 гб озу с легкостью решает самые ресурсоемкие задачи , позволяя наслаждаться быстродействием « тяжелых » приложений и игр на 5 , 5 - дюймовом дисплее . аппарат получил экран , как у ipad pro , так что картинка теперь соответствует кинематографическому стандарту .'],
       ['пион ирис ромашка рассада',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / растут у нас более 70 лет / розовые , бордовые и белые / на фото цветы 2018г / п .

In [10]:
X_train.shape

(21000, 2)

In [11]:
assert X_train[10][1] == 'продам иж планета 3 , 76 год , ( стоит на старом учёте , документы утеряны ) на ходу , хорошее состояние , все интересующие вопросы по телефону ( с родной коляской на 3 тысячи дороже ) . торга не будет .'

### BOW

Составим словарь самых часто встречающихся слов в train:

In [86]:
import collections
k = collections.Counter()
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        X_tr_split = X_train[i][j].split() # Каждую строку в title и description разбиваем на слова
        for word in X_tr_split: # Словарь самых часто встречающихся слов
            k[word] += 1

Отсортируем их по убыванию частотности и сделаем список bow_vocabulary из n=10000 самых частых слов:

In [87]:
k = list(k.items()) # Список со всеми парами ключей словаря со значениями
k.sort(key = lambda x: x[1], reverse = True) # Сортитровка по значениям (частоте) по убыванию
most_common_k = dict(k[:10000]) # Делаем словар из первых 10000 элементов

bow_vocabulary = list(most_common_k.keys())  # Делаем список только из самых частых слов (ключей этого словаря)

In [88]:
assert sorted(bow_vocabulary)[::200] == ['!', '12500', '270', '700', 'by', 'gh', 'michael', 'sonata', 'ø', 'аудиоподготовка', 'большим', 'веса', 'воспроизведения', 'габариты', 'гтд', 'джинсами', 'доступность', 'загрузки', 'зимней', 'использовался', 'квартала', 'коммуникации', 'кошки', 'лакированные', 'магазин', 'металл', 'мск', 'натуральным', 'носке', 'одному', 'отвечаем', 'пассат', 'плотно', 'покраску', 'постоянные', 'примеры', 'просьба', 'размещайте', 'репетитор', 'сантехник', 'сидения', 'современного', 'стала', 'схема', 'тон', 'удлиненная', 'фасад', 'цветами', 'шея', 'эту']

In [15]:
def text_to_bow(text: str) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common (он же bow_vocabulary) 
    указано количество его употреблений
    """ 
    
    t = text.split() # Разбиваем текст на слова
    # Считаем кол-во употреблений самых популярных слов в тексте
    return np.asarray([t.count(word) for word in bow_vocabulary])

In [16]:
assert np.allclose(np.where(text_to_bow("сдаётся уютный , тёплый гараж для стартапов в ml") != 0)[0],
                   np.array([   1,    4,   12,  565,  866, 1601, 2539, 4063]))

In [17]:
def items_to_bow(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Давайте для начала попробуем строить bow только из description товара
    # assert ниже написан для bow из description

    return np.asarray([text_to_bow(i[1]) for i in items]) # Перебираем все description всех объектов

In [18]:
assert np.allclose(np.where(items_to_bow([X_train[42]])[0] != 0),
                   np.array([   0, 1, 2, 5, 6, 7, 12, 27, 41, 49, 110,
                                189,  208,  221, 2032, 3052, 7179, 9568]),)

In [19]:
X_train_bow = items_to_bow(X_train)
X_test_bow = items_to_bow(X_test)

In [20]:
from sklearn.metrics import accuracy_score

###  Логистическая регрессия и SVC

В X_train_bow и X_test_bow много нулей, хранить их в памяти не оптимально для вычислений. Воспользуемся разреженными матрицами:

In [21]:
from scipy.sparse import csr_matrix
print('Train array in memory (raw): {:.3f} Mb'.format(X_train_bow.nbytes * 1e-6))
X_train_bow_csr = csr_matrix(X_train_bow)
print('Train array in memory (compressed): {:.3f} Mb'.format(
    (X_train_bow_csr.data.nbytes + X_train_bow_csr.indptr.nbytes + X_train_bow_csr.indices.nbytes) * 1e-6))

Train array in memory (raw): 840.000 Mb
Train array in memory (compressed): 5.765 Mb


In [22]:
print('Test array in memory (raw): {:.3f} Mb'.format(X_test_bow.nbytes * 1e-6))
X_test_bow_csr = csr_matrix(X_test_bow)
print('Test array in memory (compressed): {:.3f} Mb'.format(
    (X_test_bow_csr.data.nbytes + X_test_bow_csr.indptr.nbytes + X_test_bow_csr.indices.nbytes) * 1e-6))

Test array in memory (raw): 360.000 Mb
Test array in memory (compressed): 2.480 Mb


Логистическая регрессия:

In [23]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression(max_iter=100).fit(X_train_bow_csr, y_train)
accuracy_score(bow_model.predict(X_test_bow_csr), y_test)

assert accuracy_score(bow_model.predict(X_test_bow_csr), y_test) > 0.7

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [24]:
accuracy_score(bow_model.predict(X_test_bow_csr), y_test)

0.7045555555555556

SVM:

In [25]:
from sklearn.svm import LinearSVC
bow_model = LinearSVC(max_iter=70).fit(X_train_bow_csr, y_train)
print(accuracy_score(bow_model.predict(X_test_bow_csr), y_test))

assert accuracy_score(bow_model.predict(X_test_bow_csr), y_test) > 0.68

C:\Users\user\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6833333333333333


### Модификация признаков 

Добавим title товара в bow с произвольным весом:

напишем еще одну вспомогательную функцию items_to_bow_t - только для title

In [26]:
def items_to_bow_t(items: np.array) -> np.array:
    """ Для каждого товара возвращает вектор его bow """
    # Строим bow только из title товара

    return np.asarray([text_to_bow(i[0]) for i in items]) # Перебираем все title всех объектов

Применим данную функцию к train и test:

In [27]:
X_train_bow2 = items_to_bow_t(X_train)
X_test_bow2 = items_to_bow_t(X_test)

Прибавим к соответствующим bow-векторам для description bow-вектора для 'title' товара с некоторым весом

In [28]:
X_train_bow_td = 2*X_train_bow2 + X_train_bow 
X_test_bow_td = 2*X_test_bow2 + X_test_bow

Перед оценкой качества снова используем разреженные матрицы:

In [29]:
print('Train array in memory (raw): {:.3f} Mb'.format(X_train_bow_td.nbytes * 1e-6))
X_train_bow_csr2 = csr_matrix(X_train_bow_td)
print('Train array in memory (compressed): {:.3f} Mb'.format(
    (X_train_bow_csr2.data.nbytes + X_train_bow_csr2.indptr.nbytes + X_train_bow_csr2.indices.nbytes) * 1e-6))

Train array in memory (raw): 840.000 Mb
Train array in memory (compressed): 6.028 Mb


In [30]:
print('Test array in memory (raw): {:.3f} Mb'.format(X_test_bow_td.nbytes * 1e-6))
X_test_bow_csr2 = csr_matrix(X_test_bow_td)
print('Test array in memory (compressed): {:.3f} Mb'.format(
    (X_test_bow_csr2.data.nbytes + X_test_bow_csr2.indptr.nbytes + X_test_bow_csr2.indices.nbytes) * 1e-6))

Test array in memory (raw): 360.000 Mb
Test array in memory (compressed): 2.591 Mb


Логистическая регрессия:

In [31]:
bow_model2 = LogisticRegression(max_iter=100).fit(X_train_bow_csr2, y_train)
accuracy_score(bow_model2.predict(X_test_bow_csr2), y_test)

0.7964444444444444

SVM:

In [32]:
bow_model2 = LinearSVC(max_iter=70).fit(X_train_bow_csr2, y_train)
accuracy_score(bow_model2.predict(X_test_bow_csr2), y_test)

0.7622222222222222

Теперь нормализуем данные:

In [33]:
from sklearn.preprocessing  import normalize
X_train_bow_csrtb_norm = normalize(X_train_bow_csr2)
X_test_bow_csrtb_norm = normalize(X_test_bow_csr2)

Логистическая регрессия для нормализованных данных:

In [34]:
bow_model3 = LogisticRegression(max_iter=100).fit(X_train_bow_csrtb_norm, y_train)
accuracy_score(bow_model3.predict(X_test_bow_csrtb_norm), y_test)

0.6897777777777778

SVM для нормализованных данных:

In [35]:
bow_model3 = LinearSVC(max_iter=70).fit(X_train_bow_csrtb_norm, y_train)
accuracy_score(bow_model3.predict(X_test_bow_csrtb_norm), y_test)

0.8072222222222222

Добавление в bow title товара улучшило качество лог. регрессии и SVM по равнению с прошлым заданием.
После нормализации данных качество логистическй регрессии значительно уменьшилось, а для SVM улучшилось.

### Mystem

Используем токенизатор mystem и сравним качество:

In [36]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /bin

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
tar: Error opening archive: Failed to open 'mystem-3.0-linux3.1-64bit.tar.gz'
"cp" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [37]:
pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to c:\users\user\appdata\local\temp\pip-req-build-_ehv3r76


  Error [WinError 2] Не удается найти указанный файл while executing command git clone -q https://github.com/nlpub/pymystem3 C:\Users\user\AppData\Local\Temp\pip-req-build-_ehv3r76
Cannot find command 'git' - do you have 'git' installed and in your PATH?


Note: you may need to restart the kernel to use updated packages.


In [38]:
! pip install pymystem3

In [71]:
import copy
X_train_ms = copy.deepcopy(X_train)  # Копия токенизированных данных
X_test_ms = copy.deepcopy(X_test)
X_train_ms.shape

(21000, 2)

In [57]:
from pymystem3 import Mystem
m = Mystem()

#Меняем фунцию токенизации (через лемматизацию и прибаления '')
def preprocess2(text: str) -> str:
    return ''.join(m.lemmatize(text))

#Токенизация для X_train и X_test циклами по всем элементам
for i in range(X_train_ms.shape[0]):
    for j in range(X_train.shape[1]):
        X_train_ms[i][j] = preprocess2(X_train_ms[i][j])
for i in range(X_test_ms.shape[0]):
    for j in range(X_test.shape[1]):
        X_test_ms[i][j] = preprocess2(X_test_ms[i][j])

KeyboardInterrupt: 

In [58]:
X_train_ms[:10]

array([['сапог 46 размер новый\n', 'сапог 46 размер новый\n'],
       ['светильник потолочный swarovski\n',
        'светильник потолочный swarovski 6 штука , цена за штука . в эксплуатация 2 год , продаваться в связь со смена интерьер в квартира\n'],
       ['iphone 7 plus 128gb red красный в наличие\n',
        '/ / данный цена только для подписчик instagram : iqmac / / новый красный айфон 7 plus в наличие это элегантный и мощный смартфон , который готовый в полный мера раскрывать новый возможность ios 10 . аппарат с 4 - ядерный процессор а10 и 3 гб озу с легкость решать самый ресурсоемкий задача , позволять наслаждаться быстродействие « тяжелый » приложение и игра на 5 , 5 - дюймовый дисплей . аппарат получать экран , как у ipad pro , так что картинка теперь соответствовать кинематографический стандарт .\n'],
       ['пион ирис ромашка рассада\n',
        'пион куст 500 р ( более 10 шт )/ саженец / корень 100р / расти у мы более 70 год / розовый , бордовый и белый / на фото цветок 2

Используем CountVectorizer из sklearn, чтобы не дублировать кучу кода для bow своими руками (тут это вроде не запрещено):

для этого сначала склеим строки для title и description

In [59]:
X_train_td = [item[0] + ' ' + item[1] for item in X_train_ms]
X_test_td = [item[0] + ' ' + item[1] for item in X_test_ms]

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
cnt_vec = CountVectorizer()
X_train_CV = cnt_vec.fit_transform(X_train_td)
X_test_CV = cnt_vec.transform(X_test_td)

Логистическая регрессия:

In [61]:
bow_model4 = LogisticRegression(max_iter=100).fit(X_train_CV, y_train)
accuracy_score(bow_model4.predict(X_test_CV), y_test)

0.7946666666666666

SVM:

In [62]:
bow_model4 = LinearSVC(max_iter=70).fit(X_train_CV, y_train)
accuracy_score(bow_model4.predict(X_test_CV), y_test)

0.7874444444444444

После использования mystem качество лог регресии и SVM улучшилось благодоря лемматизации данных.

### TF-IDF 

In [65]:
data = pd.read_csv("train_subset.csv", index_col='id')
X = data[['title', 'description']].to_numpy()
y = data['Category'].to_numpy()
del data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

for i in range(X_train.shape[0]):
    for j in range(X_train.shape[1]):
        X_train[i][j] = preprocess(X_train[i][j])
for i in range(X_test.shape[0]):
    for j in range(X_test.shape[1]):
        X_test[i][j] = preprocess(X_test[i][j])

In [80]:
# Давайте для простоты считать один tf-idf для title и description.
# Для каждого слова из bow_vocabulary нужно посчитать
# в тексте скольких товаров встретилось это слово

X_train_tfidf = copy.deepcopy(X_train) # Копия токенизированных данных
X_test_tfidf = copy.deepcopy(X_test)

counter = 0
count_arr = {}

# Склеиваем title и description
X_train_tfidf = [item[0] + ' ' + item[1] for item in X_train_tfidf]
X_test_tfidf = [item[0] + ' ' + item[1] for item in X_test_tfidf]

for i in bow_vocabulary:
    for j in X_train_tfidf:
        if i in j:
            counter += 1
    count_arr.update({i: counter})
    counter = 0

In [83]:
import math

def text_to_tfidf(text: str, len_doc: int) -> np.array:
    """
    Возвращает вектор, где для каждого слова из most_common
    указан tf-idf
    """
    tf = np.zeros(10000)
    idf = np.zeros(10000)
    t = text.split()
    for i in t:
        if i in bow_vocabulary:
            tf[bow_vocabulary.index(i)] = t.count(i) / len(t)
            idf[bow_vocabulary.index(i)] = math.log(len_doc / count_arr[i])
 
    return np.array(tf*idf)

In [89]:
len_train = len(X_train_tfidf)
len_test = len(X_test_tfidf)
X_train_tf_idf = []
X_test_tf_idf = []

for i in range(len_train):
    X_train_tf_idf.append(text_to_tfidf(X_train_tfidf[i], len_train))
for i in range(len_test):
    X_test_tf_idf.append(text_to_tfidf(X_test_tfidf[i], len_test))

Перед оценкой качества снова используем разреженные матрицы:

In [91]:
X_train_tf_idf_csr = csr_matrix(X_train_tf_idf)
X_test_tf_idf_csr = csr_matrix(X_test_tf_idf)

Теперь нормализуем данные:

In [92]:
X_train_tf_idf_norm = normalize(X_train_tf_idf_csr)
X_test_tf_idf_norm = normalize(X_test_tf_idf_csr)

### Модели на TF-IDF признаках

Логистическая регрессия для нормализованных данных:

In [93]:
bow_model5 = LogisticRegression(max_iter=100).fit(X_train_tf_idf_norm, y_train)
accuracy_score(bow_model5.predict(X_test_tf_idf_norm), y_test)

0.7222222222222222

SVM для нормализованных данных:

In [94]:
bow_model5 = LinearSVC(max_iter=70).fit(X_train_tf_idf_norm, y_train)
accuracy_score(bow_model5.predict(X_test_tf_idf_norm), y_test)

0.7953333333333333

На mystem лог регрессия дала качество лучше, а СВМ практически не изменился.

### Hashing Vectorizer

In [98]:
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer = HashingVectorizer()

# Склеиваем title и description - уже делали это в X_train_tfidf и X_test_tfidf - скопируем их
X_train_td = copy.deepcopy(X_train_tfidf)
X_test_td = copy.deepcopy(X_test_tfidf)

X_train_hash_v = vectorizer.fit_transform(X_train_td)
X_test_hash_v = vectorizer.transform(X_test_td)

Оценим качество:

Логистическая регрессия

In [100]:
bow_model6 = LogisticRegression(max_iter=100).fit(X_train_hash_v, y_train)
print(accuracy_score(bow_model6.predict(X_test_hash_v), y_test))

0.7236666666666667


SVM

In [101]:
bow_model6 = LinearSVC(max_iter=70).fit(X_train_hash_v, y_train)
print(accuracy_score(bow_model6.predict(X_test_hash_v), y_test))

0.8245555555555556


На mystem лог регрессия дала качество лучше, на TF-IDF признаках примерна так же. СВМ стал еще лучше, чем на mystem и TF-IDF признаках!

### Word Vectors 

Кажому слову сопоставим какой-то эмбеддинг (вектор).

In [107]:
!wget https://www.dropbox.com/s/0x7oxso6x93efzj/ru.tar.gz

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [108]:
!tar -xzvf ru.tar.gz

tar: Error opening archive: Failed to open 'ru.tar.gz'


In [110]:
! pip install gensim

  Stored in directory: C:\Users\user\AppData\Local\pip\Cache\wheels\51\88\6c\2bcd305b87c1e62cbcbf419e782ad9fc612cd4bbb71557502d
Successfully built smart-open


In [111]:
import gensim
from gensim.models.wrappers import FastText


model = FastText.load_fasttext_format('ru.bin')

unable to import 'smart_open.gcs', disabling that module


FileNotFoundError: [Errno 2] No such file or directory: 'ru.bin'

#### Я пыталась следовать советам и все скачать, но!
### коллаб сила, юпитер могила (c) Юра Саночкин
#### модуль не устанавливается в юпитере, у меня лапки. Ниже прилагаю интуитивно написанный код, пощадите :))

In [ ]:
# Эмбеддинг предложения -- сумма эмбеддингов токенов


def sentence_embedding(sentence: str) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """

    embedding_dim = model['кек'].shape[0]
    features = np.zeros([embedding_dim], dtype='float32')
    
    for word in sentence.split():
        if word in model:
            features += model[word]
    
    return features

In [ ]:
assert np.allclose(sentence_embedding('сдаётся уютный , тёплый гараж для стартапов в ml')[::50],
                   np.array([ 0.08189847,  0.07249198, -0.15601222,  0.03782297,  0.09215296, -0.23092946]))

In [ ]:
X_train_wv = []
X_test_wv = []

# Склеиваем title и description - это было в X_train_td и X_test_td
# Применим к данным функцию entence_embedding

for i in range(len_train):
    X_train__wv[i] = sentence_embedding(X_train_dt[i])
for i in range(len_test):
    X_test_wv[i] = sentence_embedding(X_test_dt[i])

Оценим качество:
Логистическая регрессия

In [ ]:
bow_model7 = LogisticRegression(max_iter=100).fit(X_train_wv, y_train)
print(accuracy_score(bow_model7.predict(X_test_wv), y_test))

SVM:

In [ ]:
bow_model7 = LinearSVC(max_iter=70).fit(X_train_wv, y_train)
accuracy_score(bow_model7.predict(X_test_wv), y_test)

\*место для вашей молитвы чтобы это скомпилилось без ошибок\*

In [115]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://sun9-43.userapi.com/yYrkLZ1pnzSae1yBMrsDeC5FMvwmR1Xw5hVU_w/mnPfT24FnAc.jpg", width=500, height=500)